## Classifier To Detect Fake News

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import cross_val_score
from re import sub
from nltk.stem import PorterStemmer

from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [4]:
df = pd.read_csv("fake_or_real_news.csv")
df.head()

,id,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [5]:
print("******** setup **********")
stemmer = PorterStemmer()

# extract into dataframes
text = pd.DataFrame(df.text, columns=['text'])
label = df['label']

******** setup **********


In [6]:
label.head()

0    FAKE
1    FAKE
2    REAL
3    FAKE
4    REAL
Name: label, dtype: object

In [15]:
text.iloc[3,0]

"— Kaydee King (@KaydeeKing) November 9, 2016 The lesson from tonight's Dem losses: Time for Democrats to start listening to the voters. Stop running the same establishment candidates. \n— People For Bernie (@People4Bernie) November 9, 2016 If Dems didn't want a tight race they shouldn't have worked against Bernie. \n— Walker Bragman (@WalkerBragman) November 9, 2016 \nNew York Times columnist Paul Krugman, who was one of Hillary Clinton’s most outspoken surrogates during the contentious Democratic primary, blamed Clinton’s poor performance on Green Party candidate Jill Stein, who has so far received a negligible number of votes nationally, saying Stein was the Ralph Nader of 2016 in preventing a Clinton victory. The account @BerniesTeachers threw Krugman’s analysis back in his face. Your candidate was the issue. Take responsibility. https://t.co/KHyOuUSrFS \n— Teachers for Bernie (@BerniesTeachers) November 9, 2016 \nAna Navarro, a Republican who recently endorsed Hillary Clinton, sum

In [10]:
distances = euclidean_distances(docarray)
distances_df = pd.DataFrame(data=distances)
distances_df

,0,1,2,3,4,5,6,7,8,9,...,6325,6326,6327,6328,6329,6330,6331,6332,6333,6334
0,0.000000,21.748563,22.045408,21.283797,20.736441,28.740216,21.977261,20.493902,26.115130,22.847319,...,21.771541,20.518285,21.587033,19.544820,21.189620,21.563859,30.182777,27.239677,24.576411,23.216374
1,21.748563,0.000000,17.406895,17.146428,16.031220,27.037012,17.888544,15.198684,24.020824,18.894444,...,18.947295,16.733201,17.175564,13.747727,16.852300,17.262677,28.774989,25.000000,21.563859,19.287302
2,22.045408,17.406895,0.000000,17.291616,15.427249,26.683328,17.291616,14.282857,23.237900,17.549929,...,18.867962,16.881943,16.186414,12.806248,16.093477,16.522712,28.017851,24.454039,20.000000,19.570386
3,21.283797,17.146428,17.291616,0.000000,14.035669,27.147744,17.435596,14.317821,23.515952,18.734994,...,18.303005,16.248077,16.401219,12.767145,16.124515,16.673332,28.425341,24.556058,21.330729,18.867962
4,20.736441,16.031220,15.427249,14.035669,0.000000,26.305893,15.779734,12.409674,22.759613,17.146428,...,17.549929,15.459625,14.899664,10.392305,14.247807,15.779734,27.730849,24.000000,20.000000,17.464249
5,28.740216,27.037012,26.683328,27.147744,26.305893,0.000000,26.739484,26.038433,30.000000,27.748874,...,27.748874,27.221315,26.720778,25.495098,26.776856,27.147744,33.151169,30.951575,28.425341,27.694765
6,21.977261,17.888544,17.291616,17.435596,15.779734,26.739484,0.000000,14.525839,24.103942,18.681542,...,18.947295,17.320508,16.941074,13.076697,16.852300,17.146428,28.425341,24.959968,21.656408,19.798990
7,20.493902,15.198684,14.282857,14.317821,12.409674,26.038433,14.525839,0.000000,22.671568,16.124515,...,16.733201,14.798649,13.928388,7.211103,13.379088,14.035669,27.477263,23.958297,19.595918,18.083141
8,26.115130,24.020824,23.237900,23.515952,22.759613,30.000000,24.103942,22.671568,0.000000,23.790755,...,24.372115,23.173260,23.494680,22.090722,23.173260,23.000000,30.380915,28.000000,25.019992,24.186773
9,22.847319,18.894444,17.549929,18.734994,17.146428,27.748874,18.681542,16.124515,23.790755,0.000000,...,20.099751,18.520259,18.110770,15.231546,18.027756,18.083141,28.687977,25.258662,21.679483,20.856654


In [12]:
distances[:10, :10]

array([[ 0.        , 21.74856317, 22.04540769, 21.28379665, 20.73644135,
        28.74021573, 21.97726098, 20.49390153, 26.11512971, 22.84731932],
       [21.74856317,  0.        , 17.40689519, 17.1464282 , 16.03121954,
        27.03701167, 17.88854382, 15.19868415, 24.0208243 , 18.89444363],
       [22.04540769, 17.40689519,  0.        , 17.29161647, 15.42724862,
        26.68332813, 17.29161647, 14.28285686, 23.23790008, 17.54992877],
       [21.28379665, 17.1464282 , 17.29161647,  0.        , 14.03566885,
        27.14774392, 17.43559577, 14.31782106, 23.51595203, 18.734994  ],
       [20.73644135, 16.03121954, 15.42724862, 14.03566885,  0.        ,
        26.30589288, 15.77973384, 12.40967365, 22.75961335, 17.1464282 ],
       [28.74021573, 27.03701167, 26.68332813, 27.14774392, 26.30589288,
         0.        , 26.73948391, 26.03843313, 30.        , 27.74887385],
       [21.97726098, 17.88854382, 17.29161647, 17.43559577, 15.77973384,
        26.73948391,  0.        , 14.52583905

## KNN

In [16]:
print("******** prepare data **********")
new_data = []
for i in range(text.shape[0]):
    new_data.append(sub("[^a-zA-Z]", " ", text.iloc[i,0]))

lowercase_data = []
for i in range(len(new_data)):
    lowercase_data.append(new_data[i].lower())

stemmed_data = []
for i in range(len(lowercase_data)):
    words = lowercase_data[i].split()
    stemmed_words = []
    for w in words:
        stemmed_words.append(stemmer.stem(w))
    stemmed_data.append(" ".join(stemmed_words))

print("******** setup vector model **********")
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", binary = True, min_df=2, stop_words='english')
docarray = vectorizer.fit_transform(stemmed_data).toarray()

print("******** model and XV **********")
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=4)

# do the 10-fold cross validation
scores = cross_val_score(model, docarray, label, cv=10)
print("Fold Accuracies: {}".format(scores))
print("XV Accuracy: {: 6.2f}%".format(scores.mean()*100))

******** prepare data **********
******** setup vector model **********
******** model and XV **********
Fold Accuracies: [0.68346457 0.6466877  0.65299685 0.670347   0.68246445 0.69352291
 0.65086888 0.70300158 0.65876777 0.67298578]
XV Accuracy:  67.15%


## Naive Bayes

In [9]:
print("******** prepare data **********")
new_data = []
for i in range(text.shape[0]):
    new_data.append(sub("[^a-zA-Z]", " ", text.iloc[i,0]))

lowercase_data = []
for i in range(len(new_data)):
    lowercase_data.append(new_data[i].lower())

stemmed_data = []
for i in range(len(lowercase_data)):
    words = lowercase_data[i].split()
    stemmed_words = []
    for w in words:
        stemmed_words.append(stemmer.stem(w))
    stemmed_data.append(" ".join(stemmed_words))

print("******** setup vector model **********")
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer = "word", binary = True, min_df=2, stop_words='english')
docarray = vectorizer.fit_transform(stemmed_data).toarray()
coords = vectorizer.get_feature_names()
docterm = pd.DataFrame(data=docarray,columns=coords)

print("******** model and XV **********")
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()


# do the 10-fold cross validation
scores = cross_val_score(model, docarray, label, cv=10)
print("Fold Accuracies: {}".format(scores))
print("XV Accuracy: {: 6.2f}%".format(scores.mean()*100))

******** prepare data **********
******** setup vector model **********
******** model and XV **********
Fold Accuracies: [0.89606299 0.89905363 0.88328076 0.87697161 0.88151659 0.87519747
 0.87677725 0.89099526 0.89099526 0.87993681]
XV Accuracy:  88.51%


### Analysis

There is a difference in accuracy of the models, the Naive Bayes model is the one that has higher accuracy. It has 88.51% of the accuracy but the KNN model only has 67.15% of the accuracy. The naive bayes is a conditional probability model and a generative classifier which thus gives a better statistical result than the KNN Model. Overall the Naive Bayes Model generally gives a faster time result than the KNN Model.